In [1]:
import mysql.connector as sql

Setting up database connection

In [2]:
mydb = sql.connect(
    host="localhost",
    user="root",
    passwd="root",
    database="green-grid",
    port="3306"
)

mycursor = mydb.cursor()

# Query 1: Peak hour usage per meter

In [3]:
mycursor.execute('''
WITH ranked_usage AS (
  SELECT
    meter_id,
    HOUR(reading_time) AS hour_of_day,
    SUM(units_consumed) AS total_units,
    ROW_NUMBER() OVER (PARTITION BY meter_id ORDER BY SUM(units_consumed) DESC) AS rn
  FROM
    consumption_logs
  GROUP BY
    meter_id, hour_of_day
)
SELECT
  meter_id,
  hour_of_day,
  total_units
FROM
  ranked_usage
WHERE
  rn = 1;
''')
result = mycursor.fetchall()
# Print the table contents
print("PEAK HOUR USAGE PER METER:")
print(" Meter_id | Hour_of_Day | Total_Units")
print("----------|-------------|-------------")
for row in result:
    print(f" {row[0]:<8} | {row[1]:<11} | {row[2]:<1}")

PEAK HOUR USAGE PER METER:
 Meter_id | Hour_of_Day | Total_Units
----------|-------------|-------------
 1        | 18          | 394.70
 2        | 8           | 428.52
 3        | 23          | 369.80
 4        | 0           | 442.21
 5        | 17          | 387.65
 6        | 6           | 474.05
 7        | 18          | 488.68
 8        | 2           | 789.42
 9        | 9           | 422.94
 10       | 14          | 494.21
 11       | 12          | 282.96
 12       | 17          | 382.45
 13       | 19          | 445.97
 14       | 3           | 485.82
 15       | 12          | 368.28
 16       | 7           | 598.33
 17       | 10          | 333.66
 18       | 10          | 419.82
 19       | 6           | 435.41
 20       | 17          | 337.25
 21       | 7           | 453.80
 22       | 17          | 424.12
 23       | 3           | 753.15
 24       | 8           | 469.27
 25       | 15          | 391.14
 26       | 8           | 228.73
 27       | 11          | 420.03
 28  

# Query 2: Zone-wise total energy consumed

In [4]:
mycursor.execute('''
SELECT
    z.zone_id,
    z.zone_name,
    SUM(cl.units_consumed) AS total_units_consumed
FROM
    consumption_logs cl
JOIN
    meters m ON cl.meter_id = m.meter_id
JOIN
    customers c ON m.customer_id = c.customer_id
JOIN
    zones z ON c.zone_id = z.zone_id
GROUP BY
    z.zone_id, z.zone_name
ORDER BY
    z.zone_id ;
''')
result = mycursor.fetchall()
# Print the table contents
print("ZONE-WISE TOTAL ENERGY CONSUMED:")
print(" Zone_id | Zone_Name | Total_Units_Consumed")
print("---------|-----------|----------------------")
for row in result:
    print(f" {row[0]:<7} | {row[1]:<9} | {row[2]:<1}")

ZONE-WISE TOTAL ENERGY CONSUMED:
 Zone_id | Zone_Name | Total_Units_Consumed
---------|-----------|----------------------
 1       | Zone_1    | 9488.99
 2       | Zone_2    | 21745.49
 3       | Zone_3    | 15245.89
 4       | Zone_4    | 19157.86
 5       | Zone_5    | 15993.06


# Query 3: Billing summaries per customer

In [5]:
mycursor.execute('''
SELECT
    c.customer_id,
    c.customer_name,
    COUNT(b.bill_id) AS total_bills,
    SUM(b.amount) AS total_billed_amount,
    SUM(CASE WHEN b.payment_status = 'unpaid' THEN 1 ELSE 0 END) AS unpaid_bills
FROM
    billing b
JOIN
    meters m ON b.meter_id = m.meter_id
JOIN
    customers c ON m.customer_id = c.customer_id
GROUP BY
    c.customer_id, c.customer_name
ORDER BY
    customer_id ;
''')
result = mycursor.fetchall()
# Print the table contents
print("Billing Summaries per Customer:")
print(" Customer_id |       Customer_Name      | Total_Bills | Total_Billed_Amount | Unpaid_Bills")
print("-------------|--------------------------|-------------|---------------------|--------------")
for row in result:
    print(f" {row[0]:<11} | {row[1]:<24} | {row[2]:<12} | {row[3]:<19} | {row[4]:<10}")

Billing Summaries per Customer:
 Customer_id |       Customer_Name      | Total_Bills | Total_Billed_Amount | Unpaid_Bills
-------------|--------------------------|-------------|---------------------|--------------
 1           | Michelle Morales         | 3            | 7923.28             | 2         
 2           | Tanya Randall            | 3            | 3032.74             | 2         
 3           | Alicia Saunders          | 3            | 5405.68             | 2         
 4           | Heather Brown            | 3            | 5575.36             | 1         
 5           | Richard Sanchez          | 3            | 6548.63             | 2         
 6           | Whitney Kim              | 3            | 4385.41             | 1         
 7           | Helen Ramirez            | 3            | 4723.01             | 1         
 8           | Marc Gonzalez            | 3            | 7971.46             | 2         
 9           | Nicholas Cruz            | 3            | 3698.30 

Closing All Database Instances

In [6]:
mydb.close()
mycursor.close()

True